In [3]:
# langgraph and langchain installed...

In [8]:
sample_state = {
    "product_id": "P001",
    "store_id": "S001",
    "sales_quantity": 120,
    "price": 99.99,
    "promotions": 1,
    "reorder_point": 100,
    "stock_levels": 80,
    "supplier_lead_time": 5,
    "warehouse_capacity": 1000,
    "order_fulfillment_time": 2,
}



#### All agents will receive and update a central state dictionary that contains context for the current product, store, inventory, pricing, and demand.

In [ ]:
initial_state = {
    "product_id": None,
    "store_id": None,
    "date": None,
    
    "sales_quantity": None,
    "price": None,
    "promotions": None,
    "seasonality_factors": None,
    "external_factors": None,
    "demand_trend": None,
    "customer_segments": None,
    "sales_7d_avg": None,

    "stock_levels": None,
    "reorder_point": None,
    "supplier_lead_time": None,
    "order_fulfillment_time": None,
    "warehouse_capacity": None,
    "stockout_frequency": None,
    "expiry_date": None,

    "demand_forecast": None,
    "reorder_decision": None,
    "order_quantity": None,
    "suggested_price": None
}


#### We’ll write each agent as a function that takes state and returns updated state.

In [28]:
def customer_agent(state: InventoryState) -> InventoryState:
    # Use dot notation for attribute access
    print(f"[CustomerAgent] Simulating demand for {state.product_id} at Store {state.store_id}")
    
    # For this example, we set simulated_demand equal to sales_quantity or default to 0 if it's missing.
    state.simulated_demand = state.sales_quantity or 0  
    return state

In [29]:
def store_agent(state: InventoryState) -> InventoryState:
    print(f"[StoreAgent] Checking stock for {state.product_id} at Store {state.store_id}")

    stock = state.stock_levels or 0
    # Use simulated_demand that was set by CustomerAgent
    demand = state.simulated_demand or 0
    reorder_point = state.reorder_point or 0

    if stock < reorder_point:
        state.reorder_decision = True
        # A simple logic for order quantity: demand increased by 20% minus available stock
        state.order_quantity = max(0, (demand * 1.2) - stock)
    else:
        state.reorder_decision = False

    return state

In [30]:
def warehouse_agent(state: InventoryState) -> InventoryState:
    print(f"[WarehouseAgent] Processing store request for {state.product_id}")

    if state.reorder_decision:
        capacity = state.warehouse_capacity or 0
        quantity = state.order_quantity or 0
        if quantity <= capacity:
            state.order_fulfilled = True
            state.fulfillment_days = state.order_fulfillment_time or 0
        else:
            state.order_fulfilled = False
            state.fulfillment_days = None
    return state


In [31]:
def supplier_agent(state: InventoryState) -> InventoryState:
    print(f"[SupplierAgent] Notified for restocking {state.product_id}")
    if not state.order_fulfilled:
        lead_time = state.supplier_lead_time or 0
        state.restock_eta = lead_time
    return state


In [32]:
from pydantic import BaseModel
from typing import Optional

class InventoryState(BaseModel):
    product_id: Optional[str] = None
    store_id: Optional[str] = None
    date: Optional[str] = None

    sales_quantity: Optional[int] = None
    price: Optional[float] = None
    promotions: Optional[int] = None
    seasonality_factors: Optional[float] = None
    external_factors: Optional[float] = None
    demand_trend: Optional[float] = None
    customer_segments: Optional[str] = None
    sales_7d_avg: Optional[float] = None

    stock_levels: Optional[int] = None
    reorder_point: Optional[int] = None
    supplier_lead_time: Optional[int] = None
    order_fulfillment_time: Optional[int] = None
    warehouse_capacity: Optional[int] = None
    stockout_frequency: Optional[float] = None
    expiry_date: Optional[str] = None

    simulated_demand: Optional[int] = None
    reorder_decision: Optional[bool] = None
    order_quantity: Optional[float] = None
    order_fulfilled: Optional[bool] = None
    fulfillment_days: Optional[int] = None
    restock_eta: Optional[int] = None


In [33]:
from langgraph.graph import StateGraph

graph = StateGraph(state_schema=InventoryState)

# Register nodes (functions)
graph.add_node("CustomerAgent", customer_agent)
graph.add_node("StoreAgent", store_agent)
graph.add_node("WarehouseAgent", warehouse_agent)
graph.add_node("SupplierAgent", supplier_agent)

# Define edges
graph.set_entry_point("CustomerAgent")
graph.add_edge("CustomerAgent", "StoreAgent")
graph.add_edge("StoreAgent", "WarehouseAgent")
graph.add_edge("WarehouseAgent", "SupplierAgent")

# End of flow
graph.set_finish_point("SupplierAgent")

# Compile graph
agent_flow = graph.compile()


In [34]:
sample_state = {
    "product_id": "P101",
    "store_id": "S201",
    "sales_quantity": 30,
    "stock_levels": 10,
    "reorder_point": 15,
    "warehouse_capacity": 100,
    "supplier_lead_time": 5,
    "order_fulfillment_time": 2
}

result = agent_flow.invoke(sample_state)
print(result)

[CustomerAgent] Simulating demand for P101 at Store S201
[StoreAgent] Checking stock for P101 at Store S201
[WarehouseAgent] Processing store request for P101
[SupplierAgent] Notified for restocking P101
{'product_id': 'P101', 'store_id': 'S201', 'sales_quantity': 30, 'stock_levels': 10, 'reorder_point': 15, 'supplier_lead_time': 5, 'order_fulfillment_time': 2, 'warehouse_capacity': 100, 'simulated_demand': 30, 'reorder_decision': True, 'order_quantity': 26.0, 'order_fulfilled': True, 'fulfillment_days': 2}
